# Diskretna Furijeova transformacija

Sadržaj:
1. [Trigonometrijska interpolacija](#Trigonometrijska-interpolacija)
1. [Furijeova matrica](#Furijeova-matrica)
1. [Brza Furijeova transformacija - FFT](#Brza-Furijeova-transformacija---FFT)
1. [Konvolucija](#Konvolucija)

In [1]:
import numpy as np
from timeit import default_timer as timer
import numpy.random as rndm

## Trigonometrijska interpolacija

Unitarne matrice su ortogonalne matrice u odnosu na skalarni proizvod kompleksnih vektora,
$$Q^HQ=QQ^H=I.$$
Kolone (i vrste) unitarne matrice $Q\in\mathcal{M}_{n\times n}$ čine ortonormiranu bazu vektora u $\mathbb{C}^n.$ Po analogiji sa ortogonalnim matricama,	unitarne matrice predstavljaju izometrije kompleksnog vektorskog prostora $\mathbb{C}^n$ jer čuvaju standardni skalarni proizvod ovog prostora,
	$$u\cdot v=v^Hu=v^HIu=v^HQ^HQu=(Qv)^H(Qu)=(Qu)\cdot(Qv).$$

Nerednim primerom uspostvljamo vezu između diskretne Furijeove transformacije vektora vrednosti $y=\begin{bmatrix}y_0&y_1&\dots&y_{N-1}\end{bmatrix}:$
$$a_0=\dfrac{f(t)\cdot\cos(0t)}{\cos(0t)\cdot\cos(0t)}=\dfrac{1}{N}\sum_{k=0}^{N-1}y_k,\qquad a_m=\dfrac{f(t)\cdot\cos(mt)}{\cos(mt)\cdot\cos(mt)}=\dfrac{2}{N}\sum_{k=0}^{N-1}y_k\cos(mt_k),$$
$$b_m=\dfrac{f(t)\cdot\sin(mt)}{\sin(mt)\cdot\sin(mt)}=\dfrac{2}{N}\sum_{k=0}^{N-1}y_k\sin(mt_k), $$
i jedne veoma važne unitarne matrice - Furijeove matrice. 

**Primer 1.** Nek aje dat diskretan skup podataka $S_N$ koji pokazuje određena svojstva periodičnosti. Želimo podatke da interpoliramo trigonometrijskim polinomom. Zbog jednostavnosti zapisa pretpostavićemo za početak da je skup podataka koji se interpolira zadat neparnim brojem  elemenata $N=2n+1$,
$$S_{2n+1}=\left\{(t_k,y_k)\ |\ t_k=\dfrac{2k\pi}{2n+1},\qquad k=0,1,\dots,2n\right\}.$$  
Trigonometrijski interpolacioni problem podrazumeva određivanje $2n+1$ nepoznatih koeficijenata trigonometrijskog polinoma $T_n(t),$ stepena $n,$
$$
T_n(t)=a_0+\sum_{j=1}^n\Big(a_j\cos(j t)+b_j\sin(j t)\Big),
$$
koji zadovoljava interpolacione uslove $T_n(t_k)=y_k.$ Iskoristićemo Ojlerove jednakosti 
\begin{align}
&\cos\varphi+i\sin\varphi=e^{i\varphi},\qquad &&\cos\varphi-i\sin\varphi=e^{-i\varphi}\\
&\cos\varphi=\dfrac{e^{i\varphi}+e^{-i\varphi}}2,\qquad && \sin\varphi=\dfrac{e^{i\varphi}-e^{-i\varphi}}{2i}
\end{align}
za kompaktniji zapis polinoma $T_n(t):$
\begin{align}
T_n(t)&=a_0+\sum_{j=1}^n\Big(a_j\dfrac{e^{ij t}+e^{-i j t}}2+b_j\dfrac{e^{i j t}-e^{-i j t}}{2i}\Big)
=a_0+\sum_{j=1}^n\Big(a_j\dfrac{e^{ij t}e^{-i j t}}2-ib_j\dfrac{e^{i j t}-e^{-i j t}}{2}\Big)\\
&=a_0+\sum_{j=1}^n\Big(e^{ij t}\dfrac{a_j-ib_j}2+e^{-i j t}\dfrac{a_j+ib_j}{2}\Big)
\end{align}

Uvedimo ozanke $z=z(t)=e^{it},\quad c_0=a_0,\quad c_j=\dfrac{a_j-ib_j}{2},\quad c_{-j}=\dfrac{a_j+ib_j}{2},\quad j=1,2,\dots,n.$ Trigonometrijski polinom tada postaje
$$T_n(t)=c_0+\sum_{j=1}^n\left(c_j z^j+c_{-j}z^{-j}\right).$$

Interpolacioni uslovi $T_n(t_k)=y_k$ onda mogu da se zapišu u obliku:
\begin{align}
z_k&=e^{it_k}=e^{i\frac{2k\pi}{2n+1}}=\big(e^{i\frac{2\pi}{2n+1}}\big)^k=z_1^k,\qquad z_k^{2n+1}=1,\\
T_n(t_k)&=c_0+\sum_{j=1}^n\left(c_j z_k^j+c_{-j}z_k^{-j}\right)=c_0+\sum_{j=1}^n\left(c_j z_1^{kj}+c_{-j}z_1^{-kj}\right)\\
&=c_0+\sum_{j=1}^n\left(c_j z_1^{kj}+c_{-j}z_1^{(2n+1-j)k}\right)=c_0+\sum_{j=1}^n\left(c_j z_k^{j}+c_{-j}z_k^{2n+1-j}\right)\\
&=c_0+c_1z_k+c_2z_k^2+\dots+c_nz_k^n+c_{-n}z_k^{n+1}+c_{-(n-1)}z_k^{n+2}+\dots+c_{-1}z_k^{2n}.
\end{align}
Drugim rečima, problem se svodi na određivanje koeficijenata interpolacionog algebarskog polinoma 
$$P_{2n}(z)\equiv c_{2n}=\begin{bmatrix}c_0&c_1&\dots&c_n&c_{-n}&\dots&c_{-1}\end{bmatrix}^T$$ na skupu podataka $(z_k,y_k).$ Poznato je da je matrica transformacije između dva vektora $c_{2n}$ i $y$ Vandermondova matrica čvorova $V(z_0,z_1,\dots,z_{2n}),$
$$V(z_0,z_1,\dots,z_{2n})\,c_{2n}=y,\qquad c_{2n}=V(z_0,z_1,\dots,z_{2n})^{-1}y.$$

Analogni izraz se dobija i u slučaju skupa podataka $S_N$ sa parnim brojem elemenata $N=2n$. Jedina razlika je što koeficijent $b_n$ nije definisan, pa je se postavlja $b_n=0.$

## Furijeova matrica

U radnoj svesci 26OrtogonalneFunkcije pokazano je da su kolone (i vrste) matrice $V(z_0,z_1,\dots,z_{N})$ ortogonalne i norme $N,$ odnosno 
$$V(z_0,z_1,\dots,z_{N-1})^HV(z_0,z_1,\dots,z_{N-1})=V(z_0,z_1,\dots,z_{N-1})V(z_0,z_1,\dots,z_{N-1})^H=NI_N.$$
Odatle je 
$$V(z_0,z_1,\dots,z_{N-1})^{-1}=\dfrac1N V(z_0,z_1,\dots,z_{N-1})^H=\dfrac1NV(z_1^0,z_1,\dots,z_{1}^{N-1})^H.$$
Ako uvedemo oznaku $w=e^{-i\frac{2\pi}{n}}=\overline{z_1}$ Furijeova matrica je normalizovana Vandermondova matrica 
kompleksnih korena broja $1:$
$$F_n=\dfrac{1}{\sqrt{N}}V(1,w,w^2,\dots,w^{N-1})
=\dfrac{1}{\sqrt{N}}\big[w^{kj}\big]_{k,j=0}^{N-1}=\dfrac{1}{\sqrt{N}}
\begin{bmatrix}1&1&1&\dots&1\\
1&w&w^2&\dots&w^{N-1}\\
1&w^2&w^4&\dots&w^{2(N-1)}\\
\vdots&\vdots&\vdots&\ddots&\vdots\\
1&w^{N-1}&w^{2(N-1)}&\dots&w^{(N-1)(N-1)}
\end{bmatrix},$$
Furijeova matrica je simetrična unitarna matrica, tj. važi
$$F_n^HF_n=I=F_nF_n^H,\qquad\qquad F_n^T=F_n.$$

Označimo kraće Vandermondovu matricu $V_N=V(1,w,w^2,\dots,w^{N-1}).$ Zbog načina implementacije na većini hardvera i u okviru programskih paketa, proizvode  $V_Nv$ i $\dfrac1N\overline{V_N}v$  koristićemo kao diskretnu i inverznu diskretnu Furijeovu transformaciju. To ćemo kraće označavati DFT$(N,v)$ i IDFT$(N, v),$ redom. 
\begin{align}
&DFT(N,v)=V_Nv,\\
&IDFT(N,v)=V_N^{-1}v=\dfrac1N\overline{V_N}v.
\end{align}
Algoritmi kojima se sprovode efikasno izračunavanja ovih proizvoda zovemo brza Furijeova transformacija.

Ključne osobine slede iz prethodne analize: $DFT$ je linearna transformacija koja poseduje inverznu transformaciju. Za vektor realnih komponenti $DFT$ vraća vektor sa konjugovanim parocima vrednosti. Iz tog razloga nije neophodno izračunavanje celog vektora $DFT$ već samo njegove jedne polovine. Na osnovu uvedene veze između komponenti $c=DFT(y)$ i koeficijenata $a_k,b_k$ imamo
$$a_k=2{\rm Re}\big(\frac{c_k}{N}\big),\qquad b_k=-2{\rm Im}\big(\frac{c_k}{N}\big).$$
Proverićemo to na konkretnom primeru.

In [15]:
N=10
n=int(np.floor((N-1)/2))
tk=2*np.pi*np.arange(N)/N
y=np.cos(5*tk)+2*np.sin(13*tk)+0.5*rndm.randn(N)
a0=y.mean()
am=np.empty(n)
bm=np.empty(n)
for k in range(n):
    am[k]=2*np.dot(y,np.cos((k+1)*tk))/N
    bm[k]=2*np.dot(y,np.sin((k+1)*tk))/N 
print(a0,am)
print(bm)

0.013184301347158246 [-0.10343622 -0.46351581  0.22546575  0.08200447]
[-0.01218794 -0.2528891   2.27414075  0.17792719]


In [20]:
c=np.fft.fft(y)/N
np.real(c[0]),2*np.real(c[1:n+1])

(0.013184301347158246,
 array([-0.10343622, -0.46351581,  0.22546575,  0.08200447]))

In [21]:
-2*np.imag(c[1:n+1])

array([-0.01218794, -0.2528891 ,  2.27414075,  0.17792719])

## Brza Furijeova transformacija - FFT

Iza oznake FFT ne stoji jedan postupak već familija raznovrsnih algoritama kreiranih za efikasno izračunavanje različitih oblika DFT. Zainteresovani studenti mogu pogledati npr. [rad1](https://pdfs.semanticscholar.org/9cd4/d0cee6c22a65dd6e5ae51c9b6cdbaeb550df.pdf), [rad2](https://booksc.xyz/book/44878835/4c58fa), [rad3](https://booksc.xyz/book/75460622/d71d6d) ili [knjigu](https://1lib.eu/book/5402909/4ed93d). Pokazatelj značaja ovog postupka širom inženjerskih disciplina je činjenica da se odgovarajući algoritmi sve više implementiraju kao hardversko, a ne samo kao softversko rešenje.  Algoritmi koji su specijalizovani za izračunavanje diskretnih Furijeovih transformacija realnih nizova označavaju se kraće RFFT (Real Fast Fourier Transform) i IRFFT (Inverse Real Fast Fourier Transform).  U okviru kursa koristićemo univerzalnu oznaku FFT za sve navedene transformacije.

Osnovu brzih algoritama FFT čini posebno svojstvo Vandermondove matrice $V_N$ parne dimenzije $N=2n.$ Blok oblik matrice $V_N$ 
$$V_{N}P=V_{2n}P=
 \begin{bmatrix} V_n & D_nV_n\\  V_n & -D_nV_n\end{bmatrix}, $$
 gde je $P$ tzv. par-nepar permutaciona matrica,
omogućava relaksaciju u broju neophodnih aritmetičkih operacija potrebnih za izračunavanje proizvoda $V_Nv.$

**Teorema 1.** Neka je $v\in\mathbb{C}^N, N=2^n$ realan vektor. Broj potrebnih operacija množenja kompleksnih brojeva za izračunavanje proizvoda $V_Nv$ je 
$$\dfrac{N}{2}\log_2N=\mathcal{O}(N\log_2N)
=\mathcal{O}(n2^n).$$

**Primer 1.** Uporedićemo procesorsko vreme izvršenja proizvoda $Av$  i transformacije $FFT(n,v)$ za vektor $v\in\mathbb{R}^{n}$ velike dimenzije, gde je $A\in\mathcal{M}_{n\times n}$ kompleksna matrica. Matrica $A$ i vektor $v$ su slučajno generisani. Na taj način eksperiment možemo da ponovimo proizvoljan broj puta kako bismo potvrdili razliku u vremenima izvršenja. Za izračunavanje same transformacije koristićemo `numpy.fft` [paket](#https://numpy.org/doc/stable/reference/routines.fft.html) i njegove ugrađene funkcije.

In [2]:
n=2**10
A=rndm.normal(1,2,(n,n))+1j*2.3
v=rndm.normal(0,2,(n,1))

In [3]:
start = timer()
A@v
end = timer()
print("Vreme izracunavanja: ",end - start,"sec")

Vreme izracunavanja:  0.009935799999999162 sec


In [4]:
start = timer()
np.fft.fft(v)
end = timer()
print("Vreme izracunavanja: ",end - start,"sec")

Vreme izracunavanja:  0.0001458000000003068 sec


**Zadatak 1.** Izračunati DFT i IDFT sledećih vektora.
\begin{array}{rlcrl}
         a)&y=\begin{bmatrix} 1&-1&0&-1\end{bmatrix}^T && b)& y=\begin{bmatrix} 1&-1&-1&1\end{bmatrix}^T\\
         c)&y=\begin{bmatrix} 1&i&i&1\end{bmatrix}^T && d)& y=\begin{bmatrix} 1&-i&i&-1\end{bmatrix}^T.
\end{array}


**Rešenje :**

Ideju iza brzog postupka izračunavanja ovih prozvoda upoznaćemo kroz primere. Označimo sa $P$ matricu par-nepar permutacije,
$$P=\begin{bmatrix}1&0&0&0\\0&0&1&0\\0&1&0&0\\0&0&0&1\end{bmatrix}.$$

a) $y=\begin{bmatrix} 1&-1&0&-1\end{bmatrix}^T,$
\begin{align}
    {\rm DFT}(4,y)&=V_4y=V_4PP^Ty=\left[\begin{array}{cc|cc}
    1&1&1&1\\1&-1&-i&i\\\hline1&1&-1&-1\\1&-1&i&-i\end{array}\right]
    \begin{bmatrix} 1\\0\\\hline-1\\-1\end{bmatrix}\quad \Longrightarrow\\
    {\rm DFT}(2,y_{parno})&=V_2y_{parno}=\begin{bmatrix}1&1\\1&-1\end{bmatrix}
    \begin{bmatrix} 1\\0\end{bmatrix}=\begin{bmatrix} 1\\1\end{bmatrix},\\
    {\rm DFT}(2,y_{neparno})&=V_2y_{neparno}=\begin{bmatrix}1&1\\1&-1\end{bmatrix}\begin{bmatrix} -1\\-1\end{bmatrix}=
    \begin{bmatrix} -2\\0\end{bmatrix}\\
    D\,V_2\,y_{neparno}&=\begin{bmatrix}1&0\\0&-i\end{bmatrix}
    \begin{bmatrix} -2\\0\end{bmatrix}
    =\begin{bmatrix} -2\\0\end{bmatrix}\\
    {\rm DFT}(4,y)&=\begin{bmatrix} {\rm DFT}(2,y_{parno})+D\,{\rm DFT}(2,y_{neparno})\\
    {\rm DFT}(2,y_{parno})-D\,{\rm DFT}(2,y_{neparno})\end{bmatrix}=\begin{bmatrix}-1\\1\\3\\1\end{bmatrix}.
\end{align}

Inače, rezultat možemo da proverimo standardnim množenjem matrica:
\begin{align}
    V_4y&=\begin{bmatrix}1&1&1&1\\1&-i&-1&i\\1&-1&1&-1\\1&i&-1&-i\end{bmatrix}
    \begin{bmatrix} 1\\-1\\0\\-1\end{bmatrix}=\begin{bmatrix}-1\\1\\3\\1\end{bmatrix}.
\end{align}

Za inverznu transformaciju imamo:

${\rm IDFT}(y)=V_4^{-1}y=\frac14\overline{V_4}y=\dfrac14\overline{V_4\overline{y}}=4\overline{V_4y}=\frac14\begin{bmatrix}-1&1&3&1\end{bmatrix}^T.$

Naredba u Pythonu kojom se izračunava diskretna Furijeova transformacija je `numpy.fft.fft` dok je inverzna diskretna Furijeova transformacija implementirana u Pythonu kao `numpy.fft.ifft`.

In [9]:
y=np.array([1,-1,0,-1])
np.fft.fft(y)

array([-1.+0.j,  1.+0.j,  3.+0.j,  1.+0.j])

In [10]:
np.fft.ifft(y)

array([-0.25+0.j,  0.25+0.j,  0.75+0.j,  0.25+0.j])

b) $y=\begin{bmatrix} 1&-1&-1&1\end{bmatrix}^T,$
\begin{align}
V_2y_{parno}&=\begin{bmatrix}1&1\\1&-1\end{bmatrix}\begin{bmatrix} 1\\-1\end{bmatrix}
=\begin{bmatrix} 0\\2\end{bmatrix},\\
    V_2y_{neparno}&=\begin{bmatrix}1&1\\1&-1\end{bmatrix}\begin{bmatrix} -1\\1\end{bmatrix}=
    \begin{bmatrix} 0\\-2\end{bmatrix}\\ 
    DV_2y_{neparno}&=\begin{bmatrix}1&0\\0&-i\end{bmatrix}\begin{bmatrix} 0\\-2\end{bmatrix}
    =\begin{bmatrix} 0\\2i\end{bmatrix}\\
     V_4y&=\begin{bmatrix} V_2y_{parno}+DV_2y_{neparno}\\
    V_2y_{parno}-DV_2y_{neparno}\end{bmatrix}=\begin{bmatrix}0\\2+2i\\0\\2-2i\end{bmatrix}.\\
    V_4^{-1}y&=\dfrac14\overline{V_4}y=\dfrac14\overline{V_4\overline{y}}=\dfrac14\overline{V_4y}=\dfrac12\begin{bmatrix}0&1-i&0&1+i\end{bmatrix}^T.
\end{align}

In [11]:
y=np.array([1,-1,-1,1])
np.fft.fft(y)

array([0.+0.j, 2.+2.j, 0.+0.j, 2.-2.j])

In [12]:
np.fft.ifft(y)

array([0. +0.j , 0.5-0.5j, 0. +0.j , 0.5+0.5j])

c) $y=\begin{bmatrix} 1&i&i&1\end{bmatrix}^T,$
\begin{align}
V_2y_{parno}&=\begin{bmatrix}1&1\\1&-1\end{bmatrix}
\begin{bmatrix} 1\\i\end{bmatrix}=\begin{bmatrix} 1+i\\1-i\end{bmatrix},\\
    V_2y_{neparno}&=\begin{bmatrix}1&1\\1&-1\end{bmatrix}\begin{bmatrix} i\\1\end{bmatrix}=
    \begin{bmatrix} 1+i\\i-1\end{bmatrix},\\ 
    DV_2y_{neparno}&=\begin{bmatrix}1&0\\0&-i\end{bmatrix}\begin{bmatrix} 1+i\\i-1\end{bmatrix}
    =\begin{bmatrix} 1+i\\1+i\end{bmatrix}\\
     V_4y&=\begin{bmatrix} V_2y_{parno}+DV_2y_{neparno}\\
    V_2y_{parno}-DV_2y_{neparno}\end{bmatrix}=\begin{bmatrix}2+2i\\2\\0\\-2i\end{bmatrix}.
\end{align}  

Kako je matrica $V_2$ realna važiće sledeće:
\begin{align}
    V_4^{-1}y&=\dfrac14\overline{V_4}y=\dfrac14\overline{V_4\overline{y}},\\
    V_2\overline{y_{parno}}&=\overline{V_2y_{parno}}=\begin{bmatrix} 1-i\\1+i\end{bmatrix},\qquad
    V_2\overline{y_{neparno}}=\overline{V_2y_{neparno}}=
    \begin{bmatrix} 1-i\\-1-i\end{bmatrix}\\ 
    DV_2\overline{y_{neparno}}&=\begin{bmatrix}1&0\\0&-i\end{bmatrix}\begin{bmatrix} 1-i\\-1-i\end{bmatrix}
    =\begin{bmatrix} 1-i\\-1+i\end{bmatrix},\\
    V_4^{-1}y&=\dfrac14\overline{\begin{bmatrix} V_2\overline{y_{parno}}+DF_2\overline{y_{neparno}}\\
    V_2\overline{y_{parno}}-DV_2\overline{y_{neparno}}\end{bmatrix}}
    =\dfrac14\overline{\begin{bmatrix}2-2i\\2i\\0\\2\end{bmatrix}}
    =\dfrac14\begin{bmatrix}2+2i\\-2i\\0\\2\end{bmatrix}
    =\dfrac12\begin{bmatrix}1+i\\-i\\0\\1\end{bmatrix}.
\end{align}

In [13]:
y=np.array([1,1j,1j,1])
np.fft.fft(y)

array([2.+2.j, 2.+0.j, 0.+0.j, 0.-2.j])

In [14]:
np.fft.ifft(y)

array([0.5+0.5j, 0. -0.5j, 0. +0.j , 0.5+0.j ])

d) Transformacije vektora $y=\begin{bmatrix} 1&-i&i&-1\end{bmatrix}^T$ i $y=\begin{bmatrix}1&-1&0&1\end{bmatrix}^T$ izračunaćemo pomoću ugrađenih naredbi Pythona.

In [15]:
y=np.array([1,-1j,1j,-1])
np.fft.fft(y)   

array([0.+0.j, 0.-2.j, 2.+2.j, 2.+0.j])

In [16]:
np.fft.ifft(y)   

array([0. +0.j , 0.5+0.j , 0.5+0.5j, 0. -0.5j])

In [17]:
y=np.array([1,-1,0,1])
np.fft.fft(y)   

array([1.+0.j, 1.+2.j, 1.+0.j, 1.-2.j])

In [18]:
np.fft.ifft(y)   

array([0.25+0.j , 0.25-0.5j, 0.25+0.j , 0.25+0.5j])

Upoznaćemo osobine Vandermondove i Furijeove matrice koje su u tesnoj vezi sa primenama diskretene Furijeove transformacije. 

**Zadatak 3.** Za polinom $P(x)=a_0+a_1x+\dots+a_{n-1}x^{n-1}$ i $n-$te korene jedinice $w_k=e^{-i\frac{2k\pi}{n}},\ k=0,1,\dots,n-1,$ pokazati da je
$$\dfrac{1}{n}\displaystyle\sum_{k=0}^{n-1}|P(w_k)|^2=\displaystyle\sum_{k=0}^{n-1}|a_k|^2.$$

**Rešenje :**

Pre dokazivanja neke jednakosti u teoriji možemo rezultat da proverimo kroz eksperiment. Kreiraćemo slučajni vektor koeficijenata nekog polinoma i proveriti na njemu datu jednakost.

In [24]:
n=rndm.randint(20,100)  #broj koeficijenata polinoma
n

22

In [25]:
p=rndm.rand(n)
Pw=np.fft.fft(p)
leva_strana=(np.sum(np.abs(Pw)**2))/n
desna_strana=np.sum(p**2)
print(np.round(leva_strana-desna_strana))

0.0


Vektor vrednosti $p=\begin{bmatrix} P(w_0)&P(w_1)&\dots&P(w_{n-1})\end{bmatrix}^T$ dobija se Furijeovom transformacijom vektora koeficijenata $a=\begin{bmatrix} a_0&a_1&\dots&a_{n-1}\end{bmatrix}^T.$  Označimo $w=e^{-i\frac{2\pi}{n}}.$
\begin{align}
p&=\begin{bmatrix} P(w^0)\\ P(w^1)\\ \vdots \\ P(w^{n-1})\end{bmatrix}
=\begin{bmatrix} a_0+a_1w^0+a_2w^{0\cdot2}+\dots+a_{n-1}w^{0(n-1)}\\
a_0+a_1w+a_2w^2+\dots+a_{n-1}w^{n-1}\\ \vdots \\ 
a_0+a_1w^{n-1}+a_2w^{2(n-1)}+\dots+a_{n-1}w^{(n-1)^2}\end{bmatrix}=
\begin{bmatrix} 1&1&1&\dots&1\\
1&w&w^2&\dots&w^{n-1}\\ \vdots&\vdots&\vdots& &\vdots\\
1&w^{n-1}&w^{2(n-1)}&\dots&w^{(n-1)^2}\end{bmatrix}
\begin{bmatrix} a_0\\a_1\\a_2\\\vdots\\a_{n-1}\end{bmatrix}\\
&=V_na=FFT\left(n,a\right).
\end{align}

S obzirom da je $F_n=\dfrac{1}{\sqrt{n}}V_n$ unitarna matrica, odnosno izometrija kompleksnog vektorskog prostora, to je 
$$\|a\|=\|F_na\|=\left\|\dfrac{1}{\sqrt{n}}V_na\right\|=\dfrac{1}{\sqrt{n}}\|p\|.$$ 
što predstavlja traženu jednakost u vektorskom obliku. Dokazana jednakost je specijalan oblik poznate [Parsevalove jednakosti](https://en.wikipedia.org/wiki/Parseval%27s_theorem).

**Zadatak 4.** Za permutacionu matricu $P=\begin{bmatrix} 0&1&0&\dots&0\\
0&0&1&\dots&0\\ \vdots&\vdots&\vdots&\ddots&\vdots\\0&0&0&\dots&1\\1&0&0&\dots&0\end{bmatrix}\in\mathcal{M}_{n\times n}$ i Vandermondovu matricu $V_n=V(w^0,w^1,\dots,w^{n-1}),$ $w=e^{-i\frac{2\pi}{n}}$ važi 
$$ PV_n=V_nD,$$
gde je $D={\rm diag}(w^0,w^1,\dots,w^{n-1}).$ Dokazati.

**Rešenje :**

Proverićemo jednakost prvo na matrici male dimenzije.

In [26]:
n=4
I=np.eye(n)
perm=np.append(np.arange(1,n),0)
P=I[perm]
P

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [27]:
w=np.exp(-1j*2*np.pi/n)
d=w**np.arange(n)
Vn=np.vander(d,increasing=True)
print(np.round(P@Vn-(Vn*d)))

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-0.j 0.-0.j 0.-0.j]]


Označimo kolone Vandermondove matrice
$V_n=[w^{jk}]_{j,k=0}^{n-1}$ sa 
$$ kol_k=\begin{bmatrix} w^0 & w^k & w^{2k} &\dots& w^{k(n-1)}\end{bmatrix}^T,\quad k=0,1,\dots,n-1.$$

Tada za $k=0,1,\dots,n-1,$ imamo da je 
$$ P\cdot kol_{k}=
\begin{bmatrix} 0&1&0&\dots&0\\
0&0&1&\dots&0\\&&&\ddots&\\0&0&0&\dots&1\\1&0&0&\dots&0\end{bmatrix}
\begin{bmatrix} w^0 \\ w^k \\ w^{2k} \\\dots\\ w^{k(n-1)}\end{bmatrix}=
\begin{bmatrix} w^{k} \\ w^{2k} \\\dots\\ w^{k(n-1)} \\ w^0\end{bmatrix}=
\begin{bmatrix} w^{k} \\ w^{2k} \\\dots\\ w^{k(n-1)} \\ w^{kn}\end{bmatrix}=
w^k\begin{bmatrix} w^0 \\ w^k \\ w^{2k} \\\dots\\ w^{k(n-1)}\end{bmatrix}=
w^k\cdot kol_{k}.
$$

Primenjujući ovo pravilo na svaku kolonu matrice $V_n$ direktno dobijamo traženu jednakost
\begin{align}
PV_n&=\begin{bmatrix} 0&1&0&\dots&0\\
0&0&1&\dots&0\\&&&\ddots&\\0&0&0&\dots&1\\1&0&0&\dots&0\end{bmatrix}
\begin{bmatrix} 1&1&1&\dots&1\\
1&w&w^2&\dots&w^{n-1}\\ \vdots&\vdots&\vdots& &\vdots\\
1&w^{n-2}&w^{2(n-2)}&\dots&w^{(n-1)(n-2)}\\
1&w^{n-1}&w^{2(n-1)}&\dots&w^{(n-1)^2}\end{bmatrix}
=\begin{bmatrix} 
1&w&w^2&\dots&w^{n-1}\\ \vdots&\vdots&\vdots& &\vdots\\
1&w^{n-2}&w^{2(n-2)}&\dots&w^{(n-1)(n-2)}\\
1&w^{n-1}&w^{2(n-1)}&\dots&w^{(n-1)^2}
\\1&1&1&\dots&1\end{bmatrix}\\
&=\begin{bmatrix} 
1&w\cdot1&w^2\cdot1&\dots&w^{n-1}\cdot1\\ 
\vdots&\vdots&\vdots& &\vdots\\
1&w\cdot w^{n-3}&w^2\cdot w^{2(n-3)}&\dots&w^{n-1}\cdot w^{(n-1)(n-3)}\\
1&w\cdot w^{n-2}&w^2\cdot w^{2(n-2)}&\dots&w^{n-1}\cdot w^{(n-1)(n-2)}
\\1&w\cdot w^{n-1}&w^2\cdot w^{2(n-1)}&\dots&w^{n-1}\cdot w^{(n-1)^2}\end{bmatrix}\\
&=
\begin{bmatrix} 1&1&1&\dots&1\\
1&w&w^2&\dots&w^{n-1}\\ \vdots&\vdots&\vdots& &\vdots\\
1&w^{n-2}&w^{2(n-2)}&\dots&w^{(n-1)(n-2)}\\
1&w^{n-1}&w^{2(n-1)}&\dots&w^{(n-1)^2}\end{bmatrix}
\begin{bmatrix} 1&0&0&\dots&0\\
0&w&0&\dots&0\\0&0&w^{2}&\dots&0\\&&&\ddots&\\0&0&0&\dots&w^{n-1}\end{bmatrix}
=V_nD.\end{align}

Dobijena jednakost je spektralna dekompozicija matrice permutacija $P=V_nDV_n^{-1}$ i opisuje sopstvene vrednosti i sopstvene vektore matrice $P.$ Osim toga,
spektralne karakteristike matrice $P$ obezbeđuju i veoma bitnu osobinu diskretne Furijeove transformacije: 
$$
PV_n=V_n\,D.
$$

Ova jednakost, uz oznaku $c=V_ny,$ u narednom obliku poznata je kao [zakon modulacije](https://sr.wikipedia.org/sr-ec/%D0%94%D0%B8%D1%81%D0%BA%D1%80%D0%B5%D1%82%D0%BD%D0%B0_%D0%A4%D1%83%D1%80%D0%B8%D1%98%D0%B5%D0%BE%D0%B2%D0%B0_%D1%82%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D1%98%D0%B0#%D0%9F%D0%BE%D0%BC%D0%B5%D1%80%D0%B0%D1%9A%D0%B5_%D0%B8_%D1%81%D0%BA%D0%B0%D0%BB%D0%B8%D1%80%D0%B0%D1%9A%D0%B5_%D1%83_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D1%83_%D0%B8_%D1%84%D1%80%D0%B5%D0%BA%D0%B2%D0%B5%D0%BD%D1%86%D0%B8%D1%98%D0%B8).
\begin{align}
    &Pc=PV_n\,y=V_n\,D\,y\\
    \Longleftrightarrow\quad&
    \begin{bmatrix} 0&1&0&\dots&0\\
0&0&1&\dots&0\\&&&\ddots&\\0&0&0&\dots&1\\1&0&0&\dots&0\end{bmatrix}
\begin{bmatrix} c_0\\c_1\\ \vdots\\ c_{n-2}\\c_{n-1}\end{bmatrix}
=\begin{bmatrix} c_1\\c_2\\ \vdots\\ c_{n-1}\\c_0\end{bmatrix}=V_n
    \begin{bmatrix} y_0\\wy_1\\ \vdots\\ w^2y_{n-2}\\ w^{n-1}y_{n-1}\end{bmatrix},\qquad w=e^{-i\frac{2\pi}{n}}.
\end{align}

In [28]:
n=rndm.randint(50,100)
y=rndm.rand(n)
c=np.fft.fft(y)
permc=np.append(c[1:],c[0])

w=np.exp(-1j*2*np.pi/n)
d=w**np.arange(n)
cw=np.fft.fft(d*y)

print(np.round(permc-cw))

[-0.+0.j  0.+0.j -0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.-0.j -0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.-0.j -0.-0.j  0.+0.j -0.+0.j -0.+0.j -0.+0.j
 -0.-0.j  0.+0.j -0.-0.j -0.-0.j -0.-0.j  0.-0.j -0.+0.j -0.+0.j -0.-0.j
  0.-0.j  0.+0.j -0.+0.j -0.+0.j -0.-0.j  0.-0.j -0.+0.j -0.-0.j -0.-0.j
 -0.-0.j  0.-0.j  0.-0.j  0.+0.j  0.-0.j  0.+0.j  0.+0.j -0.-0.j  0.-0.j
 -0.-0.j -0.-0.j  0.-0.j  0.-0.j -0.-0.j  0.-0.j  0.-0.j  0.+0.j]


Zahvaljujući spektralnim karakteristikama Vandermondove (i Furijeove matrice) možemo da pratimo kako ciklična permutacija vrsta $P^Ty$ ulaznog vektora $y$ utiče na promenu njegove Furijeove transformacije $V_ny:$
$$V_nP^Ty=(PV_n^T)^Ty=(PV_n)^Ty=(V_n\,{D})^Ty={D}V_n y,$$
tj.
$$
V_n\begin{bmatrix} y_{n-1}\\y_0\\ \vdots\\ y_{n-3}\\y_{n-2}\end{bmatrix}=\begin{bmatrix} 1&0&0&\dots&0\\
0&w&0&\dots&0\\0&0&w^{2}&\dots&0\\&&&\ddots&\\0&0&0&\dots&w^{n-1}\end{bmatrix} V_n
\begin{bmatrix} y_0\\y_1\\ \vdots\\ y_{n-2}\\y_{n-1}\end{bmatrix}
$$
Ova jednakost poznata je kao DFT pravilo o [pomeraju u vremenskom domenu](https://sr.wikipedia.org/sr-ec/%D0%94%D0%B8%D1%81%D0%BA%D1%80%D0%B5%D1%82%D0%BD%D0%B0_%D0%A4%D1%83%D1%80%D0%B8%D1%98%D0%B5%D0%BE%D0%B2%D0%B0_%D1%82%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D1%98%D0%B0#%D0%9F%D0%BE%D0%BC%D0%B5%D1%80%D0%B0%D1%9A%D0%B5_%D0%B8_%D1%81%D0%BA%D0%B0%D0%BB%D0%B8%D1%80%D0%B0%D1%9A%D0%B5_%D1%83_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D1%83_%D0%B8_%D1%84%D1%80%D0%B5%D0%BA%D0%B2%D0%B5%D0%BD%D1%86%D0%B8%D1%98%D0%B8). 

In [29]:
permy=np.append(y[-1],y[:-1])
cpy=np.fft.fft(permy)

print(np.round(cpy-d*c))

[ 0.+0.j -0.-0.j  0.-0.j  0.+0.j  0.+0.j  0.+0.j -0.-0.j -0.+0.j -0.+0.j
 -0.-0.j  0.-0.j  0.+0.j  0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.+0.j -0.-0.j
 -0.+0.j -0.+0.j  0.-0.j -0.-0.j -0.+0.j -0.-0.j -0.+0.j  0.+0.j -0.-0.j
 -0.+0.j  0.-0.j -0.-0.j -0.+0.j -0.+0.j  0.+0.j  0.+0.j -0.-0.j -0.-0.j
  0.+0.j -0.-0.j  0.-0.j  0.+0.j  0.-0.j  0.-0.j -0.+0.j  0.+0.j  0.+0.j
  0.-0.j -0.-0.j -0.+0.j  0.+0.j  0.-0.j -0.-0.j  0.+0.j -0.+0.j]


<div class="alert alert-block alert-info">
<b>Napomena:</b>
Uvedene oznake brze Furijeove transformacije služe da prepoznajemo delove izračunavanja koja mogu biti ubrzana njenom upotrebom. Naredne jednakosti koristiti kao podsetnik u te svrhe.
\begin{align}
&V_n v\equiv FFT(n,v),\\
&V_n^{-1} v\equiv IFFT(n,v).
\end{align}
</div>

**Задатак 8.** За Вандермондову матрицу $V_n=V(w^0,w^1,\dots,w^{n-1}),$ $w=e^{-i\frac{2\pi}{n}}$ и пермутациону матрицу $$S=\begin{bmatrix} 1&0&0&\dots&0&0\\
0&0&0&\dots&0&1\\0&0&0&\dots&1&0\\ \vdots&\vdots&\vdots& &\vdots&\vdots\\0&0&1&\dots&0&0\\0&1&0&\dots&0&0\end{bmatrix}$$ важи $V_nS=\overline{V_n}.$ Доказати.

**Rešenje :**


Označimo kolone matrice $V_n$ sa $kol_k=\begin{bmatrix} w^0&w^k&w^{2k}&\dots&w^{(n-1)k}\end{bmatrix}^T,$ sa opsegom indeksa $k=0,1,\dots,n-1.$ Tada je
$$
    V_nS=\left[\begin{array}{c|c|c|c}
        \phantom{a} & \phantom{a} & \phantom{a} & \phantom{a}   \\
         kol_0& kol_1 &\dots&kol_{n-1}\\
         \phantom{a} & \phantom{a} & \phantom{a} & \phantom{a}   \\
    \end{array}\right]
    \begin{bmatrix} 1&0&\dots&0\\0&0&\dots&1\\ \vdots&\vdots& &\vdots\\0&1&\dots&0\end{bmatrix}
    =\left[\begin{array}{c|c|c|c}
        \phantom{a} & \phantom{a} & \phantom{a} & \phantom{a}   \\
         kol_0& kol_{n-1} &\dots&kol_{1}\\
         \phantom{a} & \phantom{a} & \phantom{a} & \phantom{a}   \\
    \end{array}\right].
$$

Kako je $\overline{kol_0}=\begin{bmatrix} 1&1&1&\dots&1\end{bmatrix}^T=kol_{0},$ ostaje da se pokaže da je $\overline{kol_k}=kol_{n-k},$ $k=1,\dots,n-1.$
\begin{align}
kol_{n-k}&=\begin{bmatrix} w^0&w^{n-k}&w^{2(n-k)}&\dots&w^{(n-1)(n-k)}\end{bmatrix}^T\\
&\stackrel{w^n=1}{=}\begin{bmatrix} w^0&w^{-k}&w^{2(-k)}&\dots&w^{(n-1)(-k)}\end{bmatrix}^T\\
&=\begin{bmatrix} w^0&w^{-k}&w^{-2k}&\dots&w^{-(n-1)k}\end{bmatrix}^T
=\overline{kol_k}\\
&\Longrightarrow\qquad V_nS=\overline{V_n}.
\end{align}

## Konvolucija

**Definicija 1.** Za dva vektora dužine $n:$ 
$$v=\begin{bmatrix} v_1&v_2&\dots&v_n\end{bmatrix}^T\quad\mbox{ i }\quad u=\begin{bmatrix} u_1&u_2&\dots&u_n\end{bmatrix}^T,$$ operacija množenja vektora član po član, odnosno Adamarov proizvod vektora, je data sa      
$$u\circ v=\begin{bmatrix} u_1v_1&u_2v_2&\dots&u_nv_n\end{bmatrix}^T.$$

Kao što je poznato od ranije, član-po-član množenje vektora u Numpy-u je relizovano operacijom `*`. Množenje dijagonalnom matricom svodi se na ovu operaciju. 

In [5]:
u=np.array([1,2,3,-1])
u*u

array([1, 4, 9, 1])

Ukoliko se operacija `*` primenjuje između matrice i vektora treba obratiti pažnju na rezultat. Množenje odgovarajućim koeficijentom sprovodi se duž jedne kolone.

In [6]:
A=np.ones((3,4))
A

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [7]:
A*u

array([[ 1.,  2.,  3., -1.],
       [ 1.,  2.,  3., -1.],
       [ 1.,  2.,  3., -1.]])

In [8]:
A@np.diag(u)

array([[ 1.,  2.,  3., -1.],
       [ 1.,  2.,  3., -1.],
       [ 1.,  2.,  3., -1.]])

Rezultat množenja dva polinoma, tj. izračunavanje konvolucije dva vektora, može se značajno brže sprovesti za velike stepene polinoma primenom $FFT,$ tj. algoritmima kojima je implementirano brzo izračunavanje $DFT.$ 

Za $w=e^{-i\frac{2\pi}{n+m+1}}$ i $w^k\in\sqrt[n+m+1]1,$ $k=0,1,\dots,n+m,$ poznavanje $n+m+1$ vrednosti   $P_{n+m}(w^k),$    omogućava određivanje vektora koeficijenata polinoma $P_{n+m}$ uz pomoć  $DFT$. S obzirom da je 
$$P_{n+m}(w^k)=P_{n}(w^k)\cdot P_{m}(w^k),\quad k=0,1,\dots,n+m,$$ 
za primenu $DFT$ koristimo Adamarov proizvod dva vektora, tj. član-po-član množenje. 
$$y=\begin{bmatrix}
     P_{n+m}(w^0)\\P_{n+m}(w^1)\\
     \vdots\\P_{n+m}(w^{n+m})
     \end{bmatrix}=y_1\circ y_2=
     \begin{bmatrix}
     P_{n}(w^{0})\\P_{n}(w^{1})\\
     \vdots\\P_{n}(w^{n+m})
     \end{bmatrix}\circ 
     \begin{bmatrix}
     P_{m}(w^{0})\\P_{m}(w^{1})\\
     \vdots\\P_{m}(w^{n+m})
     \end{bmatrix}.$$    

Postupak brzog množenja algebarskih polinoma sastoji se iz sledećih koraka: 
1. Dopuniti vektore koeficijenata polinoma $P_n$ i $P_m$ nulama do dužine $n+m+1,$
$$P_n\equiv a=\begin{bmatrix} a_0&a_1&\dots&a_n&0&\dots&0\end{bmatrix}^T,\quad
        P_m\equiv b=\begin{bmatrix} b_0&b_1&\dots&b_m&0&\dots&0\end{bmatrix}^T.$$
1. Izračunati $y_1=DFT(n+m+1,a)$ i $y_2=DFT(n+m+1,b).$ Drugim rečima, izvršiti sintezu vektora vrednosti dva polinoma $P_n$ i $P_m$ u tačkama $w^k\in\sqrt[n+m+1]1,$
\begin{align}
            y_1&=\begin{bmatrix} P_n(w^{0}) & P_n(w^{1}) & \dots &
            P_n(w^{n+m})\end{bmatrix}^T,\\
             y_2&=\begin{bmatrix} P_m(w^{0}) & P_m(w^{1}) & \dots &
            P_m(w^{n+m})\end{bmatrix}^T.
\end{align}
1. Pomnožiti član po član vektore vrednosti $y_1$ i $y_2$ i smestiti rezultate u vektor $y.$ Vektor $y$ predstavlja vektor vrednosti polinoma $P_{n+m}$ u tačkama $w_{k}\in\sqrt[n+m+1]1.$
\begin{align}
 y&=y_1\circ y_2\\
        &=\begin{bmatrix} P_n(w^{0})P_m(w^{0})  & P_n(w^{1})P_m(w^{1})  & \dots &
            P_n(w^{n+m})P_m(w^{n+m})\end{bmatrix}^T\\
            &=\begin{bmatrix} P_{n+m}(w^{0}) & P_{n+m}(w^{1}) & \dots &
            P_{n+m}(w^{n+m})\end{bmatrix}^T.
\end{align}          
1. Primeniti $p=IDFT(n+m+1,y)$ za dobijanje vektora koeficijenata polinoma $P_{n+m}.$

Ukoliko označimo $N=n+m+1,$ prethodno opisani postupak predstavljen je matematičkom formulom
$$p=a\ast b=IDFT\big(N,DFT(N,a)\circ DFT(N,b)\big).$$   
Time smo pokazali i veoma bitnu osobinu diskretne Furijeove transformacije
$$DFT(N,a\ast b)=DFT(N,a)\circ DFT(N,b).$$
Imajući u vidu da su
$$a=IDFT(N,y_1),\quad b=IDFT(N,y_2)\quad\mbox{ i }\qquad a\ast b=IDFT(N,y)=IDFT(N,y_1\circ y_2),$$
dobijamo  jednakost
$$IDFT(N,y_1\circ y_2)=a\ast b=IDFT(N,y_1)*IDFT(N,y_2).$$

Prikazani postupak brzog izračunavanja konvolucije može se slično primeniti i za neke druge operacije nad vektorima koje po strukturi nalikuju konvoluciji, kao što su [ciklična konvolucija vektora](https://en.wikipedia.org/wiki/Circular_convolution), [kros-korelacija vektora](https://en.wikipedia.org/wiki/Cross-correlation) i autokorelacija.

In [5]:
a=rndm.rand(2**7)
b=rndm.rand(2**12)

In [6]:
from scipy import signal

In [7]:
start = timer()
np.convolve(a,b)
end = timer()
print("Vreme izracunavanja: ",end - start,"sec")

Vreme izracunavanja:  0.00030839999817544594 sec


In [8]:
start = timer()
signal.fftconvolve(a,b)
end = timer()
print("Vreme izracunavanja: ",end - start,"sec")

Vreme izracunavanja:  0.004429900000104681 sec


In [9]:
start = timer()
signal.convolve(a,b)
end = timer()
print("Vreme izracunavanja: ",end - start,"sec")

Vreme izracunavanja:  0.0005994000021019019 sec


In [10]:
start = timer()
signal.oaconvolve(a,b)
end = timer()
print("Vreme izracunavanja: ",end - start,"sec")

Vreme izracunavanja:  0.007495199999539182 sec
